In [3]:
from Supporting_functions import *
from WBNS import WBNS_image
import RedLionfishDeconv as rl

import scipy.ndimage as ndi
from aicsimageio.readers import CziReader
from aicsimageio.writers import OmeTiffWriter
import re

from skimage import measure  

import os
import numpy as np
import tifffile
from tqdm import tqdm

def custom_sort_images(file_name):
    allparts = file_name.split('(')
    parts = allparts[1].split(')')
    return (parts[0].zfill(4))

/home/hmorales/miniconda3/envs/selfnet/lib/python3.9/site-packages/pydantic/_migration.py:281: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [33]:
# Input

## Generate initial tiffs form czi 

# Initial data extraction , if False, use pre-cropped images
extractImagesFromCzi = True                                               
czi_folder_path = '/run/user/1000/gvfs/smb-share:server=134.34.176.179,share=pmtest_fast/Daniel/SqtsfGFP_dataforHernan/forAnalysis/231218_sqtsfGFP_singletrack_18deg/'
ChannelId2Export = 1
Nimages2Skip = 5
blurWnd = 2 # for masking
thres_crop = 2.0
dataexperiment = '231218_sqtsfGFP_18deg'
tempScale = 0.309723275 #TO-DO remove # Make image isotropic from czi to remove tempScale

## Tiff source folders

# Source paths for the tiff images if extractImagesFromCzi== True, the folders will be created
srcpath  = r'/run/user/1000/gvfs/smb-share:server=134.34.176.179,share=pmtest_fast/Daniel/SqtsfGFP_dataforHernan//croppedfortraining/'
psf_path = r'/run/user/1000/gvfs/smb-share:server=134.34.176.179,share=pmtest_fast/Daniel/SqtsfGFP_dataforHernan/Averaged_transformed_PSF_561.tif'

dirOut    = r'/run/user/1000/gvfs/smb-share:server=134.34.176.179,share=pmtest_fast/Daniel/SqtsfGFP_dataforHernan/croppedfortraining/Models/Nuclei/'

dirSource = srcpath + 'Nuclei_allData/'
dirTarget = srcpath + 'Nuclei_allData_deconvolved/' 


## Generate image planes for training data 


## Process Target image
processTargetImages = False
# BG subtraction
resolution_px = 10 # FWHM of the PSF
resolution_pz = 0
noise_lvl = 2
# deconvolution
padding = 32
Niter = 10
# post processing
sigmaLoG = 0.0
sigmaLoGAddScale = 0.0
sigma = 0.8
# image normalization
thres_scale_target = 2.0 # threhold for simple mask for normalization
percentiles_target = (20, 99.995)
min_v_target = 0
max_v_target = 65535
# save processed images
dirProcTarget = srcpath + 'Nuclei_allData_deconvolved/' 

## Process Source image
# image normalization
thres_scale_source = 1.5  # threhold for simple mask for normalization
percentiles_source = (20, 99.999)
min_v_source = 0
max_v_source = 65535



## Parameters for the patch selection
patch_size = 64
stride = 64
signal_intensity_threshold = 10000  #parameter for selecting image patches containing signals

xy_interval=2
xz_interval=3

# Generate initial tiffs from czi 

In [ ]:
if extractImagesFromCzi == True:

    # Create folders

    createFolder(dirSource)
    if dirSource != dirTarget:
        createFolder(dirTarget)

    # Get all tif images in the folder
    image_names = sorted([f for f in os.listdir(czi_folder_path) if f.endswith('.czi')])
    image_names.sort(key=custom_sort_images)
    image_names = image_names[0::Nimages2Skip]


    for i, image_name in enumerate(image_names):

        start_time = time.time()  # Record the start time 

        # get image path 
        czi_file_path = os.path.join(czi_folder_path, image_name)   
        reader = CziReader(czi_file_path)


        match = re.search(r'\((\d+)\)', image_name)
        timeId = match.group(1)
        timeId= timeId.zfill(3)
        print('spim_TL'+str(timeId))

        for view in range(reader.dims.V): 
            for color in range(reader.dims.C): 

                start_time = time.time()  # Record the start time 

                # Process only the channel of interest
                if color == ChannelId2Export:

                    # Open image
                    lazy_t0 = reader.get_image_dask_data("ZYX", V=view, C=color)  # returns 3D ZYX numpy array
                    img = lazy_t0.compute()  # returns in-memory 3D numpy array
                    img = img.astype(np.uint16)
                    scale = reader.physical_pixel_sizes.X / reader.physical_pixel_sizes.Z
                    print(img.shape)

                    # Crop for calculations
                    bounds_min, bounds_max, mask = get_image_cropping_box(img, blurWnd, scale, thres_crop)          
                    img = img[bounds_min[0]:bounds_max[0], bounds_min[1]:bounds_max[1], bounds_min[2]:bounds_max[2]]

                    # Make image isotropic
                    img_shape = img.shape
                    img = reslice(img,'xy',reader.physical_pixel_sizes.X,reader.physical_pixel_sizes.Z)
                    new_img_shape = img.shape   
                    new_physical_pixel_sizeZ = img_shape[0] * reader.physical_pixel_sizes.Z / new_img_shape[0]
                    print(f"image dimension from : {img_shape} to {new_img_shape}")
                    print(f"z-space from : {reader.physical_pixel_sizes.Z} to {new_physical_pixel_sizeZ}") 

                    # Save image
                    img = img.astype(np.uint16)
                    outName = 'spim'+dataexperiment+'_TL'+str(timeId)+'_Channel'+str(color)+'_Angle'+str(view)+'.tif'
                    img_out = os.path.join(dirSource, outName)            
                    tifffile.imwrite(      
                        img_out,
                        img,
                        imagej=True, 
                        bigtiff=True,
                        resolution=(1.0/reader.physical_pixel_sizes.X, 1.0/reader.physical_pixel_sizes.Y), 
                        metadata={'spacing': new_physical_pixel_sizeZ, 'unit': 'um', 'axes': 'ZYX'})

        

        Elapsed_time = time.time() - start_time
        print(f"Elapsed Time: {Elapsed_time:.4f} seconds, image {image_name}")   

# Generate image planes for training data 

In [27]:
## Create folders to export data
outDir = dirOut+"raw_data/"
xy_data = outDir+"xy/"
xy_lr_data = outDir+"xy_lr/"
xz_data = outDir+"xz/"
   
createFolder(dirOut)
createFolder(outDir)
createFolder(xy_data)
createFolder(xy_lr_data)
createFolder(xz_data)

folder: '/run/user/1000/gvfs/smb-share:server=134.34.176.179,share=pmtest_fast/Daniel/SqtsfGFP_dataforHernan/croppedfortraining/Models/Nuclei/' already exists
folder: '/run/user/1000/gvfs/smb-share:server=134.34.176.179,share=pmtest_fast/Daniel/SqtsfGFP_dataforHernan/croppedfortraining/Models/Nuclei/raw_data/' already exists
folder: '/run/user/1000/gvfs/smb-share:server=134.34.176.179,share=pmtest_fast/Daniel/SqtsfGFP_dataforHernan/croppedfortraining/Models/Nuclei/raw_data/xz/' already exists


## Generate image planes for target and intermediate images

In [29]:
# ** Images in dirSource were manually cropped

# Get all tif images in the target folder, process and export them
image_names = sorted([f for f in os.listdir(dirTarget) if f.endswith('.tif')])

if processTargetImages == True:
    #Creat output folder
    createFolder(dirProcTarget)
    # Open PSF and Prepare PSF
    psf = tifffile.imread(psf_path)
    psf_f = psf.astype(np.float32)
    psf_norm = psf_f/psf_f.sum()

    
count = 1
for i, image_name in tqdm(enumerate(image_names)):

    start_time = time.time()  # Record the start time 
    print(f"** Processing image : {image_name}")
    
    # Open image and get metadata
    img_path = os.path.join(dirTarget, image_name)   
    img = tifffile.imread(img_path)
    img_shape = img.shape
    [physical_pixel_sizeX,physical_pixel_sizeY,physical_pixel_sizeZ] = read_tiff_voxel_size(img_path)    
    scale = physical_pixel_sizeX / physical_pixel_sizeZ
    
    
    if processTargetImages == True:
        # Get mask
        mask = get_image_simple_mask(img, sigma, scale, thres_scale_target)  
        mask =  mask.astype(np.int16)

        # Make image isotropic
        if abs(1.0-scale) > 1e-4: 
            img = reslice(img,'xy',physical_pixel_sizeX,physical_pixel_sizeZ)
        img = img.astype(np.float32)
        new_img_shape = img.shape   
        new_physical_pixel_sizeZ = img_shape[0] * physical_pixel_sizeZ / new_img_shape[0]
        print(f"     -image dimension from : {img_shape} to {new_img_shape}")
        print(f"     -z-space from : {physical_pixel_sizeZ} to {new_physical_pixel_sizeZ}")
        
        # Deconvolution
        if Niter > 0: 
            # Padding image
            img = np.pad(img, padding, mode='reflect')
            imgSizeGB = img.nbytes / (1024 ** 3)
            print('     -size(GB) : ', imgSizeGB)
            # GPU deconvolution
            res_gpu = rl.doRLDeconvolutionFromNpArrays(img, psf, niter=Niter,resAsUint8=False)
            # Removing padding
            img = res_gpu[padding:-padding, padding:-padding, padding:-padding]

        # Remove noise and BG
        if resolution_px > 0:
            img = WBNS_image(img, resolution_px, noise_lvl)
            if resolution_pz > 0:
                img_xz=np.transpose(img,[1,0,2])
                img_xz = WBNS_image(img_xz, resolution_pz, 0)
                img = np.transpose(img_xz,[1,0,2])
        # LoG filter
        if sigmaLoG > 0 :
            imgBorders = ndi.gaussian_laplace(img, sigmaLoG)
            imgBorders *= -1.0
            #imgBorders[imgBorders < 0] = 0
            imgBorders *= sigmaLoGAddScale
            img += imgBorders
        
        # Smooth
        if sigma > 0:
            img = ndi.gaussian_filter(img, sigma)
        
        # Image Normalization
        if percentiles_target[0] > 0 or percentiles_target[1] < 100:
            low_thres, high_thres0 = getNormalizationThresholds(img, percentiles_target) # low thres in whole image
            low_thres0, high_thres = getNormalizationThresholds(img * mask, percentiles_target) # high thres in FG
            img = remove_outliers_image(img, low_thres, high_thres)

        img = image_scaling(img, min_v_target, max_v_target, True)
        img = img.astype(np.uint16)
        
        # Save processed image
        img_out_name = os.path.join(dirProcTarget, image_name)            
        tifffile.imwrite(      
            img_out_name,
            img,
            imagej=True, 
            resolution=(1.0/physical_pixel_sizeX, 1.0/physical_pixel_sizeY), 
            metadata={'spacing': new_physical_pixel_sizeZ, 'unit': 'um', 'axes': 'ZYX'})
        
    # Generate intermediate images down sample and then upsample
    img_lr = np.zeros_like(img)
    z,y,x = img.shape    
    new_y = round(y * tempScale)
    new_x = round(x * tempScale)
    
    for i in range(z):
        temp_img  = cv2.resize(img[i,:,:],(new_x,new_y),interpolation=cv2.INTER_CUBIC)
        #img_lr[i,:,:] = cv2.resize(temp_img,(x,y),interpolation=cv2.INTER_CUBIC)  
        img_lr[i,:,:] = cv2.resize(temp_img,(x,y),interpolation=cv2.INTER_LINEAR) 
            
    # Export planes for target and intermediate images, each plane as a TIFF image
        
    for i in range(z):
        # Get plane stats
        outName_target = f"{xy_data}{count}.tif"
        outName_interm = f"{xy_lr_data}{count}.tif"
        tifffile.imwrite(outName_target, img[i,:,:])
        tifffile.imwrite(outName_interm, img_lr[i,:,:])      
        count += 1

    Elapsed_time = time.time() - start_time
    print(f"Elapsed Time: {Elapsed_time:.4f} seconds, image {image_name}, {count-1} images exported ") 

0it [00:00, ?it/s]

** Processing image : spim231218_sqtsfGFP_18deg_TL190_Channel1_Angle0.tif


1it [00:21, 21.77s/it]

Elapsed Time: 21.7682 seconds, image spim231218_sqtsfGFP_18deg_TL190_Channel1_Angle0.tif, 527 images exported 
** Processing image : spim231218_sqtsfGFP_18deg_TL195_Channel1_Angle0.tif


2it [00:46, 23.51s/it]

...

Elapsed Time: 24.5888 seconds, image spim231218_sqtsfGFP_18deg_TL225_Channel1_Angle0.tif, 4562 images exported 
** Processing image : spim231218_sqtsfGFP_18deg_TL230_Channel1_Angle0.tif


9it [03:44, 24.51s/it]

Elapsed Time: 24.4329 seconds, image spim231218_sqtsfGFP_18deg_TL230_Channel1_Angle0.tif, 5141 images exported 
** Processing image : spim231218_sqtsfGFP_18deg_TL235_Channel1_Angle0.tif


10it [04:07, 24.71s/it]

Elapsed Time: 22.9451 seconds, image spim231218_sqtsfGFP_18deg_TL235_Channel1_Angle0.tif, 5723 images exported 


## Generate image planes for source images

In [8]:
# Get all tif images in the dirSource folder, process and export them
image_names = sorted([f for f in os.listdir(dirSource) if f.endswith('.tif')])

count = 1
for i, image_name in tqdm(enumerate(image_names)):

    start_time = time.time()  # Record the start time 
    print(f"** Processing image : {image_name}")
    
    # Open image and get metadata
    img_path = os.path.join(dirSource, image_name)   
    img = tifffile.imread(img_path)
    img_shape = img.shape
    [physical_pixel_sizeX,physical_pixel_sizeY,physical_pixel_sizeZ] = read_tiff_voxel_size(img_path)    
    scale = physical_pixel_sizeX / physical_pixel_sizeZ
 
    # Get mask
    mask = get_image_simple_mask(img, sigma, scale, thres_scale_source)  
    mask =  mask.astype(np.int16)
    
    # Image Normalization
    if percentiles_source[0] > 0 or percentiles_source[1] < 100:
        low_thres, high_thres0 = getNormalizationThresholds(img, percentiles_source) # low thres in whole image
        low_thres0, high_thres = getNormalizationThresholds(img * mask, percentiles_source) # high thres in FG
        img = remove_outliers_image(img, low_thres, high_thres)
  
    img = image_scaling(img, min_v_source, max_v_source, True)
    img = img.astype(np.uint16)

    # reslice Image
    img_xz = reslice(img,'xz',physical_pixel_sizeX,physical_pixel_sizeZ)
    
    z,y,x = img_xz.shape 
    
    for i in range(z):
        outName_target = f"{xz_data}{count}.tif"
        tifffile.imwrite(outName_target, img_xz[i,:,:])
        count += 1
  
    Elapsed_time = time.time() - start_time
    print(f"Elapsed Time: {Elapsed_time:.4f} seconds, image {image_name}, {count-1} images exported ")    
    

0it [00:00, ?it/s]

** Processing image : spim231218_sqtsfGFP_18deg_TL190_Channel1_Angle0.tif
     -threshold_value: 379.5
Intensity Norm  from (0 , 65256) to  (0, 65535)  



1it [00:40, 40.42s/it]

Elapsed Time: 40.4249 seconds, image spim231218_sqtsfGFP_18deg_TL190_Channel1_Angle0.tif, 824 images exported 
** Processing image : spim231218_sqtsfGFP_18deg_TL195_Channel1_Angle0.tif
     -threshold_value: 376.5
Intensity Norm  from (0 , 65252) to  (0, 65535)  



2it [01:23, 41.89s/it]

...

Elapsed Time: 47.0072 seconds, image spim231218_sqtsfGFP_18deg_TL225_Channel1_Angle0.tif, 6809 images exported 
** Processing image : spim231218_sqtsfGFP_18deg_TL230_Channel1_Angle0.tif
     -threshold_value: 393.0
Intensity Norm  from (0 , 65239) to  (0, 65535)  



9it [06:40, 45.43s/it]

Elapsed Time: 44.7900 seconds, image spim231218_sqtsfGFP_18deg_TL230_Channel1_Angle0.tif, 7664 images exported 
** Processing image : spim231218_sqtsfGFP_18deg_TL235_Channel1_Angle0.tif
     -threshold_value: 391.5
Intensity Norm  from (0 , 65245) to  (0, 65535)  



10it [07:30, 45.09s/it]

Elapsed Time: 50.3011 seconds, image spim231218_sqtsfGFP_18deg_TL235_Channel1_Angle0.tif, 8519 images exported 


# Generate training data from image planes

In [30]:
# Create folder output 
train_data_path = os.path.join(dirOut, 'train_data/')
createFolder(train_data_path)

folder: '/run/user/1000/gvfs/smb-share:server=134.34.176.179,share=pmtest_fast/Daniel/SqtsfGFP_dataforHernan/croppedfortraining/Models/Nuclei/train_data/' already exists


In [34]:
# Initialize arrays

xy = []
xy_lr = []
xz = []

# Loop over lateral images
file_list_xy = os.listdir(xy_data)
for i in tqdm(range(0, len(file_list_xy), xy_interval)):
    xy_img = tifffile.imread(xy_data + str(i + 1) + '.tif')
    xy_lr_img = tifffile.imread(xy_lr_data + str(i + 1) + '.tif')
    L0 = min(xy_img.shape[0], xy_lr_img.shape[0])
    L1 = min(xy_img.shape[1], xy_lr_img.shape[1])
    for m in range(0, L0 - patch_size + 1, stride):
        for n in range(0, L1 - patch_size + 1, stride):
            crop_xy    =    xy_img[m:m + patch_size, n:n + patch_size]
            crop_xy_lr = xy_lr_img[m:m + patch_size, n:n + patch_size]
            
            if np.max(crop_xy) >= signal_intensity_threshold:
                xy.append(crop_xy)
                xy_lr.append(crop_xy_lr)

# Loop over axial images   
file_list_xz = os.listdir(xz_data)
for i in tqdm(range(0, len(file_list_xz), xz_interval)):
    xz_img = tifffile.imread(xz_data + str(i + 1) + '.tif')
    for m in range(0, xz_img.shape[0] - patch_size + 1, stride):
        for n in range(0, xz_img.shape[1] - patch_size + 1, stride):
            crop_xz = xz_img[m:m + patch_size, n:n + patch_size]

            if np.max(crop_xz) >= signal_intensity_threshold:
                xz.append(crop_xz)


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2840/2840 [00:36<00:00, 77.93it/s]


In [35]:
print(len(xy))
print(len(xy_lr))
print(len(xz))


96492
96492
87231


In [36]:
# Convert to arays and save

xy = np.array(xy, dtype=np.float32)
xy_lr = np.array(xy_lr, dtype=np.float32)
xz = np.array(xz, dtype=np.float32)
print(xy.shape, xy_lr.shape, xz.shape)

np.savez(os.path.join(train_data_path, 'train_data.npz'), xy=xy, xy_lr=xy_lr, xz=xz)

(96492, 64, 64) (96492, 64, 64) (87231, 64, 64)


In [37]:
#Save tiff to double check

import tifffile as tiff
tiff.imwrite(os.path.join(train_data_path,'xy.tif'), xy)
tiff.imwrite(os.path.join(train_data_path,'xy_lr.tif'), xy_lr)
tiff.imwrite(os.path.join(train_data_path,'xz.tif'), xz)